In [63]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from textwrap import shorten
import requests

from bs4 import BeautifulSoup

In [64]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [65]:
df = pd.read_csv('Condo_buyout_map_template_popup_pin - Sheet1.csv',dtype='str')

In [66]:
df = df.fillna('Not Available')

In [67]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['full_address'].apply(geocode)

In [68]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Image URL Collector

In [69]:
df['image_links'] = ''
links = []
for url in df['story_link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find('img',{'class':'MainArtImage_image__2YtGm'})
        images = images['src']
        links.append(images)
    except:
        links.append('no_image')

df['image_links'] = links

In [70]:
# print(df['image_links'].iloc[0])

In [71]:
# df.at[20,'image_img']=('<img src="Aimco.jpg" width="256" height="156">')
df['image_links'].iloc[1]

'https://static.therealdeal.com/wp-content/uploads/2023/07/Mast-Capital-Sued-Over-Miami-Beach-Condo-Buyout_Main-1400x875.jpg'

In [72]:
# df.at[4,'image_links']=('https://therealdeal.com/wp-content/uploads/2019/10/trd-logo-16x9.jpg')

In [73]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0    <a href="https://therealdeal.com/miami/2023/05...
1    <a href="https://therealdeal.com/miami/2023/07...
2    <a href="https://therealdeal.com/miami/2023/03...
3    <a href="https://therealdeal.com/miami/2023/06...
4    <a href="https://therealdeal.com/miami/2023/05...
5    <a href="https://therealdeal.com/miami/2023/06...
6    <a href="https://therealdeal.com/miami/2022/10...
Name: description_link, dtype: object

In [83]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    name = df['name'].iloc[i]
    buyers = df['buyers'].iloc[i]
    price = df['price'].iloc[i]
    status = df['status'].iloc[i]
    legal_c = df['currently facing legal challenge?'].iloc[i]
    image = df['image_links'].iloc[i]
    description = df['description_link'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Name: </strong>{}'''.format(name) + '''<br>
    <strong>Buyers: </strong>{}'''.format(buyers) + '''<br>
    <strong>Price: </strong>{}'''.format(price) + '''<br>
    <strong>Status: </strong>{}'''.format(status) + '''<br>
    <strong>Currently Legally Challenged: </strong>{}'''.format(legal_c) + '''<br>
    {}'''.format(description) + '''<br>
    </html>
    '''
    return html

In [84]:
import branca

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=8)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Condo Buyouts')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [85]:
m.save('index.html')

## Map URL Snagger

In [82]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/SoFla_condo_buyouts
